# IMPORTING LIBRARIES

In [2]:
import numpy as np 
import pandas as pd

In [3]:
order=pd.read_excel("Company X - Order Report (1).xlsx") 
pincode=pd.read_excel("Company X - Pincode Zones (1).xlsx") 
product=pd.read_excel("Company X - SKU Master (1).xlsx") 
invoice=pd.read_excel("Courier Company - Invoice (1).xlsx") 
rates=pd.read_excel("Courier Company - Rates (1).xlsx")

In [4]:
order.head(1)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0


In [5]:
pincode.head(1)

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d


In [6]:
product.head(1)

,SKU,Weight (g)
0,8904223815682,210


In [7]:
invoice.head(1)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0


In [8]:
rates.head(1)

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.5,29.5,23.6,13.6,23.6


# DATA CLEANING

In [9]:
order.drop_duplicates(inplace=True)

In [10]:
product.drop_duplicates(inplace=True)

In [11]:
pincode.drop_duplicates(inplace=True)

In [12]:
invoice.drop_duplicates(inplace=True)

In [13]:
rates.drop_duplicates(inplace=True)

In [14]:
order[order.duplicated(keep=False)]

,ExternOrderNo,SKU,Order Qty


In [15]:
product[product.duplicated(keep=False)]

,SKU,Weight (g)


In [16]:
pd.merge(product,order,on="SKU",how="inner")

,SKU,Weight (g),ExternOrderNo,Order Qty
0,8904223815682,210,2001808801,1.0
1,8904223815859,165,2001820978,1.0
2,8904223815859,165,2001811363,1.0
3,8904223815859,165,2001808801,1.0
4,8904223815859,165,2001808675,1.0
...,...,...,...,...
393,8904223819512,210,2001806733,1.0
394,8904223819543,300,2001811306,1.0
395,SACHETS001,10,2001827036,1.0
396,SACHETS001,10,2001825261,1.0


In [17]:
order.shape

(398, 3)

In [18]:
product.shape

(65, 2)

In [19]:
rates["Zone"]=rates["Zone"].str.lower()

# MERGING DATA

In [20]:
comp=pd.merge( pd.merge( invoice,pd.merge( pincode,rates,on="Zone",how="inner"), 
                        on="Customer Pincode",how="inner"),
              pd.merge( product,order,on="SKU",how="inner"),
              left_on="Order ID",right_on="ExternOrderNo",how="inner")

In [21]:
comp.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Warehouse Pincode_y,Zone_y,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,SKU,Weight (g),ExternOrderNo,Order Qty
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,121003,d,1.5,45.4,44.8,41.3,44.8,8904223818645,137,2001806232,6.0
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,121003,d,1.5,45.4,44.8,41.3,44.8,8904223819147,240,2001806232,2.0
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,121003,d,1.5,45.4,44.8,41.3,44.8,8904223818706,127,2001806273,1.0
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,121003,d,1.5,45.4,44.8,41.3,44.8,8904223818850,240,2001806273,1.0
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,121003,d,1.5,45.4,44.8,41.3,44.8,8904223818942,133,2001806273,1.0


In [22]:
comp["Actual Weight(kG)"]= (comp["Weight (g)"]*comp["Order Qty"])/1000

In [23]:
awb_data=comp.groupby("AWB Code").agg({"Order ID":"first","Charged Weight":"first","Warehouse Pincode_x":"first",
                             "Customer Pincode":"first","Type of Shipment":"first",
                              "Billing Amount (Rs.)":"first","Zone_y":"first","Weight Slabs":"first",
                              "Forward Fixed Charge":"first","Forward Additional Weight Slab Charge":"first",
                              "RTO Fixed Charge":"first","RTO Additional Weight Slab Charge":"first",
                              "SKU":"count","Order Qty":"sum","Actual Weight(kG)":"sum"})

In [24]:
awb_data.head(1)

,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Type of Shipment,Billing Amount (Rs.),Zone_y,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,SKU,Order Qty,Actual Weight(kG)
AWB Code,,,,,,,,,,,,,,,
1091117221940,2001806210,2.92,121003,140604,Forward charges,174.5,b,1.0,33.0,28.3,20.5,28.3,2,2.0,0.22


In [25]:
awb_data["Slab_count"]=np.ceil(awb_data["Actual Weight(kG)"]/awb_data["Weight Slabs"])

In [26]:
awb_data.head(1)

,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Type of Shipment,Billing Amount (Rs.),Zone_y,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,SKU,Order Qty,Actual Weight(kG),Slab_count
AWB Code,,,,,,,,,,,,,,,,
1091117221940,2001806210,2.92,121003,140604,Forward charges,174.5,b,1.0,33.0,28.3,20.5,28.3,2,2.0,0.22,1.0


In [27]:
def actual_price(x):
    if x["Type of Shipment"]=="Forward charges":
        return x["Slab_count"]*x["Forward Additional Weight Slab Charge"]+x["Forward Fixed Charge"]
    elif x["Type of Shipment"]=="Forward and RTO charges":
        return (x["Slab_count"]*x["Forward Additional Weight Slab Charge"]+x["Forward Fixed Charge"]
               )+(x["Slab_count"]*x["RTO Additional Weight Slab Charge"]+x["RTO Fixed Charge"])
    else:
        return "Error"

In [28]:
awb_data["Actual_Price"]=awb_data.apply(lambda x: actual_price(x),axis=1)

In [29]:
sorted(list(awb_data["Actual_Price"]))

[61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 61.3,
 89.6,
 89.6,
 89.6,
 89.6,
 89.6,
 89.6,
 89.6,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.19999999999999,
 90.199999999999

In [30]:
awb_data["Actual vs invoice"]= awb_data["Actual_Price"]-awb_data["Billing Amount (Rs.)"]

In [31]:
awb_data["Actual vs invoice"]=awb_data["Actual vs invoice"].astype(int)

In [32]:
awb_data

,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Type of Shipment,Billing Amount (Rs.),Zone_y,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,SKU,Order Qty,Actual Weight(kG),Slab_count,Actual_Price,Actual vs invoice
AWB Code,,,,,,,,,,,,,,,,,,
1091117221940,2001806210,2.92,121003,140604,Forward charges,174.5,b,1.0,33.0,28.3,20.5,28.3,2,2.0,0.220,1.0,61.3,-113
1091117222065,2001806226,0.68,121003,723146,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,1,2.0,0.480,1.0,90.2,0
1091117222080,2001806229,0.71,121003,421204,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,3,3.0,0.500,1.0,90.2,0
1091117222124,2001806232,1.30,121003,507101,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8,2,8.0,1.302,1.0,90.2,-44
1091117222135,2001806233,0.78,121003,263139,Forward charges,61.3,b,1.0,33.0,28.3,20.5,28.3,2,2.0,0.245,1.0,61.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091121666133,2001823564,0.70,121003,492001,Forward and RTO charges,172.8,d,1.5,45.4,44.8,41.3,44.8,3,6.0,0.672,1.0,176.3,3
1091121844806,2001811475,0.50,121003,173212,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,4,4.0,0.689,1.0,112.1,79
1091121846136,2001811305,0.50,121003,302020,Forward charges,45.4,b,1.0,33.0,28.3,20.5,28.3,4,4.0,0.750,1.0,61.3,15


In [33]:
def find(x):
    if x["Actual vs invoice"]<0:
        return "under charged"
    elif x["Actual vs invoice"]==0:
        return "correctly charged"
    else:
        return "over charged"

In [34]:
awb_data["Status"]=awb_data.apply(lambda x: find(x),axis=1)

In [35]:
awb_data.groupby("Status").agg({"Order ID":"count","Actual vs invoice":"sum"})

,Order ID,Actual vs invoice
Status,,
correctly charged,27,0
over charged,34,920
under charged,63,-3663


In [36]:
int(list(awb_data.groupby("Status").agg({"Order ID":"count","Actual vs invoice":"sum"}).values[1:2,1:2])[0])

920